In [112]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import numpy as np

# .env 파일에서 KAKAO_API_KEY를 로드
load_dotenv()
KAKAO_API_KEY = os.getenv('KAKAO_API_KEY')

# 독립문: 37.575020, 126.957221
# 성수JC: 37.542640, 127.029530

# 서울 중구의 대략적인 경계 좌표 (minX, minY, maxX, maxY)
# min_x, min_y = 126.973, 37.550
# max_x, max_y = 127.020, 37.574

min_x, min_y = 126.957221, 37.542640
max_x, max_y = 127.029530, 37.575020

# 5000개의 작은 영역으로 분할
num_divisions = int(np.sqrt(1000))
x_points = np.linspace(min_x, max_x, num_divisions)
y_points = np.linspace(min_y, max_y, num_divisions)

def search_places(rect):
    url = "https://dapi.kakao.com/v2/local/search/keyword.json"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    params = {
        "query": "중구",
        "rect": rect,
        "size": 15  # 한 페이지에 가져올 결과 수 (최대 15)
    }

    places = []
    page = 1
    max_pages = 3  # 최대 페이지 수

    while page <= max_pages:
        params["page"] = page
        response = requests.get(url, headers=headers, params=params)
        data = response.json()

        if "documents" not in data or not data["documents"]:
            break

        places.extend(data["documents"])

        if data["meta"]["is_end"]:
            break

        page += 1

    return places

def save_to_csv(places, filename):
    df = pd.DataFrame(places)
    df.to_csv(filename, index=False, encoding='utf-8-sig')

all_places = []
for i in range(len(x_points) - 1):
    for j in range(len(y_points) - 1):
        rect = f"{x_points[i]},{y_points[j]},{x_points[i+1]},{y_points[j+1]}"
        places = search_places(rect)
        all_places.extend(places)

save_to_csv(all_places, "before/places_1000_1000.csv")

In [113]:
df = pd.read_csv('before/places.csv')
df = df.drop_duplicates(subset=['id'])
df = df.drop(['distance','category_group_code','id'],axis=1)
df = df.dropna()
len(df)

4144

In [114]:
# '관광명소' '문화시설' 
# '숙박'
# '음식점' '카페'

df_touristattraction= df[(df['category_group_name']=='관광명소') | (df['category_group_name']=='문화시설')]
df_accommodation = df[df['category_group_name']=='숙박']
df_restaurant = df[(df['category_group_name']=='음식점') | (df['category_group_name']=='카페')]

In [115]:
df_restaurant

,address_name,category_group_name,category_name,phone,place_name,place_url,road_address_name,x,y
20,서울 중구 만리동2가 273,카페,음식점 > 카페 > 커피전문점 > 메가MGC커피,02-3147-2444,메가MGC커피 서울만리점,http://place.map.kakao.com/1853524775,서울 중구 만리재로 175,126.962852,37.552159
21,서울 중구 만리동2가 273,음식점,음식점 > 간식 > 아이스크림 > 배스킨라빈스,02-364-3331,배스킨라빈스 만리센트럴점,http://place.map.kakao.com/960690898,서울 중구 만리재로 175,126.962507,37.551916
22,서울 중구 만리동2가 226-1,음식점,음식점 > 한식 > 죽 > 본죽,02-363-6288,본죽 만리점,http://place.map.kakao.com/2132333184,서울 중구 만리재로 175,126.963017,37.552279
23,서울 중구 만리동2가 273,음식점,"음식점 > 양식 > 스테이크,립",02-313-6692,스탠딩스테이크 공덕점,http://place.map.kakao.com/148925726,서울 중구 만리재로 175,126.962949,37.552215
25,서울 중구 만리동2가 273,카페,음식점 > 카페 > 커피전문점 > 이디야커피,02-363-6848,이디야커피 만리동고개점,http://place.map.kakao.com/1039040428,서울 중구 만리재로 175,126.962768,37.552089
...,...,...,...,...,...,...,...,...,...
13546,서울 중구 황학동 2545,음식점,음식점 > 분식 > 바르다김선생,02-2048-5789,바르다김선생 황학롯데캐슬점,http://place.map.kakao.com/27440473,서울 중구 청계천로 400,127.023110,37.570909
13553,서울 중구 황학동 2545,음식점,음식점 > 분식 > 떡볶이 > 국대떡볶이,02-2253-7780,국대떡볶이 이마트청계천점,http://place.map.kakao.com/24212000,서울 중구 청계천로 400,127.022858,37.571049
13559,서울 중구 황학동 2545,카페,음식점 > 카페 > 테마카페 > 무인카페,1661-1167,비트박스 이마트청계천점,http://place.map.kakao.com/1715041279,서울 중구 청계천로 400,127.022911,37.571045
13596,서울 중구 신당동 855,음식점,"음식점 > 일식 > 초밥,롤",02-2234-7730,상무초밥 왕십리점,http://place.map.kakao.com/1354722386,서울 중구 왕십리로 407,127.024916,37.564846


In [116]:
# 불러오기
import pandas as pd
import numpy as np

df_restaurant.loc[df['category_group_name'] == '카페', 'category_group_name'] = '음식점'
df_touristattraction.loc[df['category_group_name'] == '음식점', 'category_group_name'] = '카페'
# columns 의미 -> row 전이
def df_preprocessing(df):
    df.loc[:, 'place_name'] = "장소이름:" + df['place_name']
    df.loc[:, 'phone'] = "전화번호:" + df['phone']
    df.loc[:, 'x'] = "경도:" + df['x'].astype(str)
    df.loc[:, 'y'] = "위도:" + df['y'].astype(str)
    df.loc[:, 'address_name'] = "지번:" + df['address_name']
    df.loc[:, 'road_address_name'] = "도로명주소:" + df['road_address_name']
    df.loc[:, 'place_url'] = "웹링크:" + df['place_url']
    return df

df_restaurant = df_preprocessing(df_restaurant)
df_accommodation = df_preprocessing(df_accommodation)
df_touristattraction = df_preprocessing(df_touristattraction)


df_restaurant.to_csv('after/restaurant_data.csv', index=False, encoding='utf-8')
df_accommodation.to_csv('after/accommodation.csv', index=False, encoding='utf-8')
df_touristattraction.to_csv('after/touristattraction.csv', index=False, encoding='utf-8')